In [281]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import pandas as pd
from Points import Points
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from itertools import chain
import mediapipe as mp
import glob
import os
import cv2
import random

In [4]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cuda device


In [101]:
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_face_mesh = mp.solutions.face_mesh

# For static images:
photo_landmark_dict = {}
list_paths = []
shapes = []

# CK+ load
for emotion in ["anger", "contempt", "disgust", "fear", "happiness", "neutral", "sadness", "surprise"]:
    emo_folder_path = "../Datasets/CK+/" + emotion
    for pic_file in glob.glob(os.path.join(emo_folder_path, "*png")):
        list_paths.append([emotion, pic_file])

# JAFFE load
for pic_file in glob.glob(os.path.join("../Datasets/JAFFE", "*tiff")):
    if "AN" in pic_file[16:]:
        list_paths.append(["anger", pic_file])
    elif "DI" in pic_file[16:]:
        list_paths.append(["disgust", pic_file])
    elif "FE" in pic_file[16:]:
        list_paths.append(["fear", pic_file])
    elif "HA" in pic_file[16:]:
        list_paths.append(["happiness", pic_file])
    elif "NE" in pic_file[16:]:
        list_paths.append(["neutral", pic_file])
    elif "SA" in pic_file[16:]:
        list_paths.append(["sadness", pic_file])
    else:
        list_paths.append(["surprise", pic_file])

# FER-2013
FER_emo_dict = {0:"anger", 1:"disgust", 2:"fear", 3:"happiness", 4:"neutral", 5:"sadness", 6:"surprise"}
for set in ["test/", "train/"]:
    root = "../Datasets/FER2013/" + set
    for index, emotion in enumerate(["angry", "disgust", "fear", "happy", "neutral", "sad", "surprise"]):
        emo_folder_path = root + emotion
        for pic_file in glob.glob(os.path.join(emo_folder_path, "*jpg")):
            list_paths.append([FER_emo_dict[index], pic_file])

In [293]:
angie = 0
disgust = 0
fe = 0
ha = 0
ne = 0
sa = 0
su = 0
list_ = []
#["anger", "contempt", "disgust", "fear", "happiness", "neutral", "sadness", "surprise"]:
for pic_file in glob.glob(os.path.join("../Datasets/JAFFE/", "*tiff")):
    if "AN" in pic_file[16:]:
        angie +=1
        list_.append(0)
    elif "DI" in pic_file[16:]:
        disgust +=1
        list_.append(2)
    elif "FE" in pic_file[16:]:
        fe +=1
        list_.append(3)
    elif "HA" in pic_file[16:]:
        ha +=1
        list_.append(4)
    elif "NE" in pic_file[16:]:
        ne +=1
        list_.append(5)
    elif "SA" in pic_file[16:]:
        sa +=1
        list_.append(6)
    else:
        su +=1
        list_.append(7)
print(angie, disgust, fe, ha, ne, sa, su)
print(list_)

30 29 32 31 30 31 30
[0, 0, 0, 2, 2, 2, 3, 3, 3, 3, 4, 4, 4, 4, 5, 5, 5, 6, 6, 6, 7, 7, 7, 0, 0, 0, 2, 2, 2, 2, 3, 3, 3, 4, 4, 4, 5, 5, 5, 6, 6, 6, 7, 7, 7, 0, 0, 0, 2, 2, 3, 3, 3, 4, 4, 4, 4, 5, 5, 5, 6, 6, 6, 6, 7, 7, 7, 0, 0, 0, 2, 2, 2, 3, 3, 3, 4, 4, 5, 5, 5, 6, 6, 6, 7, 7, 7, 0, 0, 0, 2, 2, 2, 3, 3, 3, 4, 4, 4, 5, 5, 5, 6, 6, 6, 7, 7, 7, 0, 0, 0, 2, 2, 2, 3, 3, 3, 4, 4, 4, 5, 5, 5, 6, 6, 6, 7, 7, 7, 0, 0, 0, 2, 2, 3, 3, 3, 4, 4, 4, 5, 5, 5, 6, 6, 6, 7, 7, 7, 0, 0, 0, 2, 2, 2, 3, 3, 3, 4, 4, 4, 5, 5, 5, 6, 6, 6, 7, 7, 7, 0, 0, 0, 2, 2, 2, 3, 3, 3, 4, 4, 4, 5, 5, 5, 6, 6, 6, 7, 7, 7, 0, 0, 0, 2, 2, 2, 3, 3, 3, 3, 4, 4, 4, 5, 5, 5, 6, 6, 6, 7, 7, 7]


In [ ]:
skipped = 0

drawing_spec = mp_drawing.DrawingSpec(thickness=1, circle_radius=1)
with mp_face_mesh.FaceMesh(
    static_image_mode=True,
    max_num_faces=1,
    refine_landmarks=True,
    min_detection_confidence=0.2) as face_mesh:
    for idx, [emo, file] in enumerate(list_paths):
        image = cv2.imread(file, 0)
        # Convert the BGR image to RGB before processing.
        results = face_mesh.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))

        if results.multi_face_landmarks is not None:
            photo_landmark_dict[idx-skipped] = [emo]
            x_coords = []
            y_coords = []
            z_coords = []
            for count, mark in enumerate(results.multi_face_landmarks[0].landmark):
                x_coords.append(mark.x)
                y_coords.append(mark.y)
                z_coords.append(mark.z)

                # MUST CROP PHOTO FIRST AND THEN STUFF.
            shape = image.shape

            x_coords = [x * shape[0] for x in x_coords]
            y_coords = [y * shape[1] for y in y_coords]

            range_x = max(x_coords) - min(x_coords)
            range_y = max(y_coords) - min(y_coords)
            offset_x = [x - min(x_coords) for x in x_coords]
            offset_y = [y - min(y_coords) for y in y_coords]
            new_x = [x / range_x for x in offset_x]
            new_y = [y / range_y for y in offset_y]

            photo_landmark_dict[idx-skipped][1:479] = zip(new_x, new_y, z_coords)
        else:
            skipped += 1
        if idx % 200 == 0:
            print(idx)
            print(f"length: {len(photo_landmark_dict[idx-skipped])}")


print(skipped)
#print(len(photo_landmark_dict[0]))

In [103]:
'''for a in photo_landmark_dict.keys():
    if len(photo_landmark_dict[a]) != 479:
        print(a)
        print(f"weird thing: {photo_landmark_dict[a]}")'''
        
dataframe = pd.DataFrame(data=photo_landmark_dict)
#print(dataframe.head())
#print(dataframe.tail())
dataframe.to_json("./photo_landmark_list.json", orient='columns')

LINE BREAK :)

In [304]:
emotion_dict = {"anger": 0, "contempt": 1, "disgust": 2, "fear": 3, "happiness": 4, "neutral": 5, "sadness": 6,"sad": 6,
            "surprise": 7}

def read_data(input_landmarks, label_points, desync, z=False):
    '''Path for json with '''
    
    emotions = np.asarray([emotion_dict[y] for y in input_landmarks.iloc[0].astype(object)], dtype=np.float32)
    columns = input_landmarks.shape[1]
    points = []
    for photo in range(columns):
        for point in label_points:
            if z is True:
                coords = input_landmarks[photo+desync].iloc[point+1]
            else:
                coords = input_landmarks[photo+desync].iloc[point+1][0:2]
                
            points.extend(coords)
    if z is True:
        points = np.array(points, dtype=np.float32).reshape(columns, len(label_points*3)) # EDIT THINGS HERE
    else:
        points = np.array(points, dtype=np.float32).reshape(columns, len(label_points*2)) # EDIT THINGS HERE


    print(f"shape at the end: {points.shape}")
    return emotions, points
    
def select_points(emotions, points, training_ratio):
    X_train, X_test, y_train, y_test = train_test_split(points, emotions, 
                                                        test_size=(1-training_ratio), random_state=1234)
    
    X_train = torch.from_numpy(X_train)
    X_test = torch.from_numpy(X_test)
    
    ''' new_y_train = np.zeros([y_train.shape[0], 7])
    for index in range(y_train.shape[0]):
        if int(y_train[index]) != emotion_dict["neutral"]:
            new_y_train[index][int(y_train[index])] = 1
    
    new_y_test = np.zeros([y_test.shape[0], 7])
    for index in range(y_test.shape[0]):
        if int(y_test[index]) != emotion_dict["neutral"]:
            new_y_test[index][int(y_test[index])] = 1'''
        
    
    y_train = torch.from_numpy(y_train)
    y_test = torch.from_numpy(y_test)
    
    
    return X_train, X_test, y_train, y_test
    

In [202]:
# 1)
min_wanted_points = Points.right_eye_middle.value + Points.left_eye_middle.value + Points.nose.value + Points.mouth_inner.value
max_wanted_points = min_wanted_points + Points.right_eye_inner.value + Points.right_eye_outer.value + \
        Points.left_eye_inner.value + Points.left_eye_outer.value
complete = list(range(468))

In [ ]:
landmarks = pd.read_json("./photo_landmark_list.json")

In [248]:
CK = landmarks.truncate(before=0, after=1132, axis="columns")
JAFFE = landmarks.truncate(before=1133, after=1345, axis="columns")
CK_JAFFE = landmarks.truncate(before=0, after=1345, axis="columns")
#print(JAFFE)
FER = landmarks.truncate(before=1345, after=35835, axis="columns")

In [324]:
landmarks[1133:1135].shape #.iloc(1133,1346)


(0, 35836)

In [289]:
FER_y, FER_x = read_data(FER, complete, 1345)

TypeError: can only concatenate str (not "int") to str

In [266]:
CK_y, CK_x = read_data(CK, complete, 0)

shape at the end: (1133, 936)


In [305]:
JAFFE_y, JAFFE_x = read_data(JAFFE, complete, 1133)

shape at the end: (213, 936)


In [250]:
CK_JAFFE_y, CK_JAFFE_x = read_data(CK_JAFFE, complete, 0)

In [250]:
all_x, all_y = read_data(landmarks, complete, 0)

shape at the end: (1346, 936)


In [234]:
X_train, X_test, y_train, y_test = select_points(emotions, points, .95)

shape at the end: (34491, 936)


In [132]:
print(y_train.shape)

torch.Size([1076])


In [262]:
# Neural Net model

# 1) Model
# Linear model f = wx + b , sigmoid at the end
class Model(nn.Module):
    def __init__(self, n_input_features):
        super(Model, self).__init__()
        self.n = n_input_features
        self.fc1 = nn.Linear(n_input_features, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, 84)
        self.fc4 = nn.Linear(84, 8)

    def forward(self, x):
        #x.reshape(1, self.n)
        #x = torch.flatten(x, 1)
        _X = 0
        '''x = self.t(self.fc1(x))
        x = self.t(self.fc2(x))
        x = self.t(self.fc3(x))'''
        # F.relu
        #nn.tanh
        # torch.sigmoid
        
        x = torch.tanh(self.fc1(x))
        x = torch.tanh(self.fc2(x))
        x = torch.tanh(self.fc3(x))
        
        x = self.fc4(x)
        return x

In [263]:
model = Model(936)
dict_ = torch.load("models/FER-model.pt")
model.load_state_dict(dict_)

model.eval()

Model(
  (fc1): Linear(in_features=936, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=256, bias=True)
  (fc3): Linear(in_features=256, out_features=84, bias=True)
  (fc4): Linear(in_features=84, out_features=8, bias=True)
  (t): Threshold(threshold=0.5, value=1)
)

In [273]:
# 2) Loss and optimizer
num_epochs = 25
learning_rate = 0.01
criterion = nn.CrossEntropyLoss()
#criterion = nn.L1Loss(size_average=None, reduce=None, reduction='mean')
#optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
optimizer = torch.optim.SGD(model.parameters(), lr=0.00005, momentum=0.3)
#optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
#optimizer = torch.optim.SGD(model.parameters(), lr=0.00033, momentum=0)


In [ ]:
# 3) Training loop

for epoch in range(num_epochs):
    running_loss = 0
    for batch, row in enumerate(X_train):

        optimizer.zero_grad()

        # Forward pass and loss
        y_pred = model(row)
        #y_pred = torch.from_numpy(np.reshape(y_pred.detach().numpy(), (1,7)))
        y_pred = torch.reshape(y_pred, (1,8))
        
        y_target = torch.tensor([int(y_train[batch])])
        torch.reshape(y_target, (-1,))
       
        loss = criterion(input=y_pred, target=y_target)

        # Backward pass and update
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        
        if batch % 1000 == 999:
            print(f'[{epoch + 1}, {batch + 1}] loss: {running_loss / 1000:.3f}')
            running_loss = 0


In [14]:
def init_model(path):
    model = Model(268)
    dict = torch.load(path)
    model.load_state_dict(dict)

    model.eval()

    return model

In [66]:
np.set_printoptions(suppress=True)


In [275]:
check_acc(model, torch.from_numpy(CK_x), torch.from_numpy(CK_y))

[[     15.       0.      16.       0.       0.       1.       1.       1.]
 [      0.       0.       0.       0.       0.       0.       0.       0.]
 [      0.       0.       0.       0.       0.       0.       0.       0.]
 [      0.       0.       3.       8.       0.       2.       0.       6.]
 [      5.       6.       8.      23.      94.      23.       1.       4.]
 [     53.      12.      59.      17.       6.     594.      54.       2.]
 [      0.       0.       0.       0.       0.       0.       0.       0.]
 [1000002.       0.       2.       9.       0.       3.       3.     100.]]
811 | accuracy of the ndetwork on the test images: 71.57987643424536 %
Accuracy for class: anger is 20.0 % | 15 out of 75
Accuracy for class: contempt is 0.0 % | 0 out of 18
Accuracy for class: disgust is 0.0 % | 0 out of 88
Accuracy for class: fear  is 14.0 % | 8 out of 57
Accuracy for class: happiness is 94.0 % | 94 out of 100
Accuracy for class: neutral is 95.3 % | 594 out of 623
Accuracy for 

In [306]:
correct_jaffe_list = [0, 0, 0, 2, 2, 2, 3, 3, 3, 3, 4, 4, 4, 4, 5, 5, 5, 6, 6, 6, 7, 7, 7, 0, 0, 0, 2, 2, 2, 2, 3, 3, 3, 4, 4, 4, 5, 5, 5, 6, 6, 6, 7, 7, 7, 0, 0, 0, 2, 2, 3, 3, 3, 4, 4, 4, 4, 5, 5, 5, 6, 6, 6, 6, 7, 7, 7, 0, 0, 0, 2, 2, 2, 3, 3, 3, 4, 4, 5, 5, 5, 6, 6, 6, 7, 7, 7, 0, 0, 0, 2, 2, 2, 3, 3, 3, 4, 4, 4, 5, 5, 5, 6, 6, 6, 7, 7, 7, 0, 0, 0, 2, 2, 2, 3, 3, 3, 4, 4, 4, 5, 5, 5, 6, 6, 6, 7, 7, 7, 0, 0, 0, 2, 2, 3, 3, 3, 4, 4, 4, 5, 5, 5, 6, 6, 6, 7, 7, 7, 0, 0, 0, 2, 2, 2, 3, 3, 3, 4, 4, 4, 5, 5, 5, 6, 6, 6, 7, 7, 7, 0, 0, 0, 2, 2, 2, 3, 3, 3, 4, 4, 4, 5, 5, 5, 6, 6, 6, 7, 7, 7, 0, 0, 0, 2, 2, 2, 3, 3, 3, 3, 4, 4, 4, 5, 5, 5, 6, 6, 6, 7, 7, 7]

nn = model

correct = 0
total = 0
classes = ["anger", "contempt", "disgust", "fear", "happiness", "neutral", "sadness", "surprise"]
# prepare to count predictions for each class
correct_pred = {classname: 0 for classname in classes}
total_pred = {classname: 0 for classname in classes}

identity = np.zeros((8,8))

with torch.no_grad():
    outputs = nn(torch.from_numpy(JAFFE_x))
    _, predicted = torch.max(outputs.data, 1)
    total += len(correct_jaffe_list)
    correct += (predicted == torch.from_numpy(np.array(correct_jaffe_list))).sum().item()
    identity[7][0] = 1000000
    for val, cor in zip(predicted, (torch.from_numpy(np.array(correct_jaffe_list)))):
        #print(int(val), int(correct))
        identity[int(val)][int(cor)] += 1
        if int(val) == int(cor):
            correct_pred[classes[int(cor)]] += 1
        total_pred[classes[int(cor)]] += 1

print(identity)
print(correct, f'| accuracy of the ndetwork on the test images: {100 * correct / total} %') # 1133, 213, 


# print accuracy for each class
for classname, correct_count in correct_pred.items():
    if total_pred[classname] != 0:
        accuracy = 100 * float(correct_count) / total_pred[classname]
        print(f'Accuracy for class: {classname:5s} is {accuracy:.1f} %', f"| {correct_count} out of {total_pred[classname]}")


[[      6.       0.      11.       9.       7.       7.      13.       8.]
 [      0.       0.       0.       0.       0.       0.       0.       0.]
 [      0.       0.       0.       0.       0.       0.       0.       0.]
 [      1.       0.       2.       0.       0.       0.       2.       0.]
 [      5.       0.       4.       6.       3.       5.       5.       6.]
 [      5.       0.       7.       9.       9.       9.       3.       8.]
 [      8.       0.       4.       6.      10.       6.       3.       6.]
 [1000005.       0.       1.       2.       2.       3.       5.       2.]]
23 | accuracy of the ndetwork on the test images: 10.7981220657277 %
Accuracy for class: anger is 20.0 % | 6 out of 30
Accuracy for class: disgust is 0.0 % | 0 out of 29
Accuracy for class: fear  is 0.0 % | 0 out of 32
Accuracy for class: happiness is 9.7 % | 3 out of 31
Accuracy for class: neutral is 30.0 % | 9 out of 30
Accuracy for class: sadness is 9.7 % | 3 out of 31
Accuracy for class: surp

In [346]:
check_acc(model, torch.from_numpy(FER_x), torch.from_numpy(FER_y))

C:\Users\Jonathan\.conda\envs\pytorch\lib\site-packages\ipykernel_launcher.py:30: RuntimeWarning: invalid value encountered in double_scalars


[[0.26116222        nan 0.32122371 0.08938433 0.01975392 0.04179643
  0.06635237 0.02846252]
 [0.         0.         0.         0.         0.         0.
  0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.        ]
 [0.03455239        nan 0.01720841 0.04377173 0.01219099 0.01442386
  0.02566789 0.02617535]
 [0.15010096        nan 0.13193117 0.1378605  0.82639124 0.09375512
  0.1246726  0.09682338]
 [0.28831052        nan 0.31739962 0.35815095 0.08940061 0.60973611
  0.37890693 0.18094028]
 [0.19632039        nan 0.13001912 0.20944979 0.02991308 0.17816751
  0.35847739 0.04066074]
 [0.06955351        nan 0.08221797 0.1613827  0.02235015 0.06212096
  0.04592282 0.62693774]]
[[0.26116221673771595, nan, 0.32122370936902483, 0.08938433217426876, 0.019753922564623546, 0.041796426815276184, 0.06635236598568185, 0.028462515883100382], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.03455238949966345, nan, 0.

In [345]:
def check_acc(model, x, y):
    nn = model
    
    correct = 0
    total = 0
    classes = ["anger", "contempt", "disgust", "fear", "happiness", "neutral", "sadness", "surprise"]
    # prepare to count predictions for each class
    correct_pred = {classname: 0 for classname in classes}
    total_pred = {classname: 0 for classname in classes}

    identity = np.zeros((8,8))

    with torch.no_grad():
        outputs = nn(x)
        _, predicted = torch.max(outputs.data, 1)
        total += y.size(0)
        correct += (predicted == y).sum().item()
        identity[7][0] +=0
        for val, cor in zip(predicted, y):
            #print(int(val), int(correct))
            identity[int(val)][int(cor)] += 1
            if int(val) == int(cor):
                correct_pred[classes[int(cor)]] += 1
            total_pred[classes[int(cor)]] += 1
    
    
    for row in range(8):
        for item in range(8):
            if correct_pred[classes[row]] != 0:
                identity[row][item] /= total_pred[classes[item]]
    print(identity)
    
    print(identity.tolist())
    print(correct, f'| accuracy of the ndetwork on the test images: {100 * correct / total} %') # 1133, 213, 



    # print accuracy for each class
    for classname, correct_count in correct_pred.items():
        if total_pred[classname] != 0:
            accuracy = 100 * float(correct_count) / total_pred[classname]
            print(f'Accuracy for class: {classname:5s} is {accuracy:.1f} %', f"| {correct_count} out of {total_pred[classname]}")

In [255]:
# Loading

# Neural Net model

# 1) Model
# Linear model f = wx + b , sigmoid at the end
class Model(nn.Module):
    def __init__(self, n_input_features):
        super(Model, self).__init__()
        self.n = n_input_features
        self.fc1 = nn.Linear(n_input_features, 400)
        self.fc2 = nn.Linear(400, 120)
        self.fc3 = nn.Linear(120, 84)
        self.fc4 = nn.Linear(84, 7)
        
        self.t = nn.Threshold(.5, 1)

    def forward(self, x):
        #x.reshape(1, self.n)
        #x = torch.flatten(x, 1)
        _X = 0
        '''x = self.t(self.fc1(x))
        x = self.t(self.fc2(x))
        x = self.t(self.fc3(x))'''
        # F.relu
        #nn.tanh
        # torch.sigmoid
        
        x = torch.tanh(self.fc1(x))
        x = torch.tanh(self.fc2(x))
        x = torch.tanh(self.fc3(x))
        
        x = self.fc4(x)
        return x

model = init_model("./models/v4-model.pt")

In [278]:
torch.save(model.state_dict(), "./models/FER2-model.pt")

# 